In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
# importar eps
df_eps = gpd.read_file("../2_Procesamiento/CapasEditadas/EspaciosPublicos_v11.geojson")
df_eps = df_eps.loc[df_eps.evp == 1]
df_eps['rango'] = '0 - 5'

In [3]:
# importar capas de municipios y límite antrópico
gdf_muni = gpd.read_file("../1_Datos/ACUMAR/LIMITE_POLITICO.shp")
gdf_antro = gpd.read_file("../1_Datos/ACUMAR/LIMITE_ANTROPICO_POLIGONO.geojson")
# filtro de los municipios de interés 
gdf_muni = gdf_muni.loc[(gdf_muni.DISTRITO == "LOMAS DE ZAMORA") | (gdf_muni.DISTRITO == "LANUS")]

In [4]:
# importar calles
df_nac = gpd.read_file("./Calles/RedVialNacional.geojson")
df_pro = gpd.read_file("./Calles/RedVialProvincial.geojson")
df_ter = gpd.read_file("./Calles/RedVialTerciaria.geojson")
df_urb = gpd.read_file("./Calles/RedVialUrbana.geojson")

KeyboardInterrupt: 

In [23]:
df_urb['hct'] = df_urb['objeto']
df_calles = pd.concat([df_nac[['hct', 'geometry']], df_pro[['hct', 'geometry']], df_ter[['hct', 'geometry']], df_urb[['hct', 'geometry']]])
df_calles.loc[df_calles.hct == 2, 'hct'] = 'Provincial'
df_calles.loc[df_calles.hct == 4, 'hct'] = 'Terciario'
print(df_calles.hct.unique())
df_calles.head()

['Nacional' 'Provincial' 'Terciario' 'Calle']


,hct,geometry
0,Nacional,"MULTILINESTRING ((-54.49869 -25.66899, -54.498..."
1,Nacional,"MULTILINESTRING ((-54.46205 -25.71356, -54.462..."
2,Nacional,"MULTILINESTRING ((-54.08179 -25.72037, -54.082..."
3,Nacional,"MULTILINESTRING ((-53.74518 -26.05548, -53.745..."
4,Nacional,"MULTILINESTRING ((-53.74290 -26.05752, -53.743..."


In [25]:
df_calles_muni = gpd.overlay(df_calles, gdf_muni, how="intersection")
df_calles_muni.head()

,hct,SUPERFICIE,DISTRITO,geometry
0,Provincial,88,LOMAS DE ZAMORA,"MULTILINESTRING ((-58.45688 -34.78142, -58.446..."
1,Provincial,88,LOMAS DE ZAMORA,"LINESTRING (-58.43914 -34.80495, -58.43024 -34..."
2,Provincial,88,LOMAS DE ZAMORA,"LINESTRING (-58.43980 -34.80533, -58.43914 -34..."
3,Provincial,88,LOMAS DE ZAMORA,"LINESTRING (-58.33941 -34.75048, -58.34284 -34..."
4,Provincial,88,LOMAS DE ZAMORA,"LINESTRING (-58.34642 -34.75396, -58.37181 -34..."


In [27]:
#df_calles_muni.to_file("./Calles/RedVial_LanusLomas.geojson", driver='GeoJSON')
df_calles_muni.to_file("./Calles/RedVial_LanusLomas.shp")

# Functions

In [9]:
def dissolveTiempos(df_time, df_ep, outpath):
    # load time data
    df_pol = df_time
    # filter ep polygons
    df_eps = df_ep.loc[df_ep.evp == 1]
    df_eps['rango'] = '0 - 5'
    # merge eps and service areas
    df_pol = pd.concat([df_pol, df_eps])
    df_pol.reset_index(inplace=True, drop=True)

    # split data by time range
    df_pol_5 = df_pol.loc[df_pol.rango == '0 - 5']
    df_pol_5['tiempo'] = '0 - 5'
    df_pol_10 = df_pol.loc[df_pol.rango == '5 - 10']
    df_pol_10['tiempo'] = '5 - 10'
    df_pol_15 = df_pol.loc[df_pol.rango == '10 - 15']
    df_pol_15['tiempo'] = '10 - 15'

    # Dissolve polygons by time range
    df_pol_5_d = df_pol_5.dissolve(by='rango')
    df_pol_10_d = df_pol_10.dissolve(by='rango')
    df_pol_15_d = df_pol_15.dissolve(by='rango')
    # erase lower times to higher times 
    df_pol_10_d_dif = gpd.overlay(df_pol_10_d, df_pol_5, how='difference')
    df_pol_15_d_dif = gpd.overlay(df_pol_15_d, df_pol_5, how='difference')
    df_pol_15_d_dif = gpd.overlay(df_pol_15_d_dif, df_pol_10_d, how='difference')
    # merge all time polygons
    df_pol_time = pd.concat([df_pol_5_d, df_pol_10_d_dif, df_pol_15_d_dif])
    df_pol_time.reset_index(inplace=True, drop=True)
    df_pol_time
    # clip to the muni area
    df_time_muni = gpd.overlay(df_pol_time, gdf_muni, how='intersection')
    # clip to the antro area
    df_time_muni_antro = gpd.overlay(df_time_muni[['Name', 'FromBreak', 'ToBreak', 'tiempo', 'DISTRITO', 'geometry']], gdf_antro[['geometry']], how='intersection')
    # export
    df_time_muni_antro.to_file(outpath)

# After Network Dataset in ArcMap

In [2]:
# importar tiempos de viaje
df_pol = gpd.read_file("./evp_acce/geonet_poltime_2.shp")
df_pol['rango'] = df_pol.apply(lambda row: row.Name.split(': ')[1], axis=1)
df_pol.rango.unique()

array(['0 - 15', '0 - 10', '0 - 7,61950549423523', '0 - 6,0169798759962',
       '0 - 5'], dtype=object)

In [3]:
df_pol.loc[df_pol.rango == '0 - 7,61950549423523', 'rango'] = '0 - 10'
df_pol.loc[df_pol.rango == '0 - 6,0169798759962', 'rango'] = '0 - 10'

df_pol.loc[df_pol.rango == '0 - 10', 'rango'] = '5 - 10'
df_pol.loc[df_pol.rango == '0 - 15', 'rango'] = '10 - 15'

df_pol.rango.unique()

array(['10 - 15', '5 - 10', '0 - 5'], dtype=object)

In [ ]:
dissolveTiempos(df_pol, df_eps, "./evp_acce/geonet_poltime_distritos_antropico.shp")

## Clean data

In [16]:
# importar time travel
df_pol = gpd.read_file("./acce_output_2/geonet_poltime_Parque.shp")
df_pol['rango'] = df_pol.apply(lambda row: row.Name.split(': ')[1], axis=1)
df_pol.rango.unique()
df_pol.to_file("./acce_output_2/geonet_poltime_Parque_clean.shp")

In [17]:
# importar time travel
df_pol = gpd.read_file("./acce_output_2/geonet_poltime_Parque_con_equipamiento.shp")
df_pol['rango'] = df_pol.apply(lambda row: row.Name.split(': ')[1], axis=1)
df_pol.rango.unique()

array(['10 - 15', '5 - 10', '5 - 8,86072503011333', '0 - 5',
       '0 - 0,565751806879071', '0 - 0,349764172997331'], dtype=object)

In [18]:
df_pol.loc[df_pol.rango == '0 - 0,349764172997331', 'rango'] = '0 - 10'
df_pol.loc[df_pol.rango == '0 - 0,565751806879071', 'rango'] = '0 - 10'
df_pol.loc[df_pol.rango == '5 - 8,86072503011333', 'rango'] = '5 - 10'

df_pol.loc[df_pol.rango == '0 - 10', 'rango'] = '5 - 10'
df_pol.loc[df_pol.rango == '0 - 15', 'rango'] = '10 - 15'

df_pol.rango.unique()

array(['10 - 15', '5 - 10', '0 - 5'], dtype=object)

In [19]:
df_pol.to_file("./acce_output_2/geonet_poltime_Parque con equipamiento_clean.shp")

In [20]:
# importar time travel
df_pol = gpd.read_file("./acce_output_2/geonet_poltime_Plaza.shp")
df_pol['rango'] = df_pol.apply(lambda row: row.Name.split(': ')[1], axis=1)
df_pol.rango.unique()
df_pol.to_file("./acce_output_2/geonet_poltime_Plaza_clean.shp")

In [21]:
# importar time travel
df_pol = gpd.read_file("./acce_output_2/geonet_poltime_Plazoleta.shp")
df_pol['rango'] = df_pol.apply(lambda row: row.Name.split(': ')[1], axis=1)
df_pol.rango.unique()
df_pol.to_file("./acce_output_2/geonet_poltime_Plazoleta_clean.shp")

In [ ]:
# importar eps
df_eps_all = gpd.read_file("../2_Procesamiento/CapasEditadas/EspaciosPublicos_v11.geojson")

for eptipo in ['Plaza', 'Plazoleta', 'Parque con equipamiento', 'Parque']:
    # load time data
    path_load = "./acce_output_2/geonet_poltime_"+eptipo+"_clean.shp"
    df_pol = gpd.read_file(path_load)
    # filter ep polygons
    df_eps = df_eps_all.loc[df_eps_all.tipo == eptipo]

    dissolveTiempos(df_pol, df_eps, "./evp_acce/geonet_poltime_"+eptipo+".shp")

# Parques Potenciales

In [9]:
# importar parques potenciales
df_parques_potenciales = gpd.read_file("../2_Procesamiento/ep_parques_potenciales/EspaciosPublicos_PARQUE CON POT.shp")

In [5]:
# importar accesibilidad a parques potenciales
df_pol = gpd.read_file("./parques_potenciales/geonet_poltime_ParquePotenciales.shp")
df_pol['rango'] = df_pol.apply(lambda row: row.Name.split(': ')[1], axis=1)
df_pol.rango.unique()

array(['10 - 15', '5 - 10', '0 - 5'], dtype=object)

In [14]:
dissolveTiempos(df_pol, df_parques_potenciales, "./parques_potenciales/geonet_poltime_ParquePotenciales_disseolve.shp")

c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


# Sistemas de EVP

In [5]:
# filter sistems
df_eps_sist = df_eps.loc[df_eps.sis_bool == 1]

In [6]:
# importar accesibilidad a sistemas
df_pol = gpd.read_file("./evp_sistemas/geonet_sistemas_ttravel_2.shp")
df_pol['rango'] = df_pol.apply(lambda row: row.Name.split(': ')[1], axis=1)
df_pol.rango.unique()

array(['10 - 15', '5 - 10', '5 - 8,86072503011333', '0 - 5',
       '0 - 0,565751806879071', '0 - 0,349764172997331'], dtype=object)

In [7]:
# clean range
df_pol.loc[df_pol.rango == '5 - 8,86072503011333', 'rango'] = '5 - 10'
df_pol.loc[df_pol.rango == '0 - 0,565751806879071', 'rango'] = '0 - 5'
df_pol.loc[df_pol.rango == '0 - 0,349764172997331', 'rango'] = '0 - 5'
df_pol.rango.unique()

array(['10 - 15', '5 - 10', '0 - 5'], dtype=object)

In [10]:
dissolveTiempos(df_pol, df_eps_sist, "./evp_sistemas/geonet_sistemas_ttravel_disseolve.shp")

c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
